# Инсталляции

In [26]:
!pip install docxtpl

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.6 MB/s eta 0:00:00
  Created wheel for docxcompose: filename=docxcompose-1.4.0-py3-none-any.whl size=23264 sha256=6db90261b5457f2c38928cdf6878c4f034b692ec8e3b25572c234d47623e6900
  Stored in directory: /root/.cache/pip/wheels/28/0e/78/2dc28f2d5b192d1a3ec237ec60e9d65654a078dfe07ce3a714
Successfully built docxcompose


In [27]:
!pip install python-docx

In [28]:
!pip install Spire.Doc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 12.3 MB/s eta 0:00:00


# Импорт

In [29]:
import os

import pandas as pd
import numpy as np

from random import shuffle, random, randint

In [30]:
# Word Documents Templating
from docxtpl import DocxTemplate

In [31]:
import json

In [32]:
from datetime import datetime, date

In [33]:
from PIL import Image, ImageFilter

In [34]:
from pathlib import Path
import requests

In [35]:
import kagglehub

# Функции генерации данных

In [36]:
def get_names():
  # Download latest version
  path = kagglehub.dataset_download("rai220/russian-cyrillic-names-and-sex")

#  print("Path to dataset files:", path)
  df_names = pd.read_csv(path + '/data.csv')
  return df_names


In [37]:
def get_cities():
  url = ("https://raw.githubusercontent.com/"
        "epogrebnyak/ru-cities/main/assets/towns.csv")

  # save file locally
  p = Path("towns.csv")
  if not p.exists():
      content = requests.get(url).text
      p.write_text(content, encoding="utf-8")

  # read as dataframe
  df_cities = pd.read_csv("towns.csv")
  #print(df.sample(5))

  return df_cities

In [38]:
def get_rand_pser(n = 4):

    ser = randint(1, (10**n-1))
    nser = str(ser).rjust(n, '1')
    return nser

In [39]:
def get_rand_pnum(n = 6):

    num = randint(0, (10**n-1))
    pnum = str(num).rjust(n, '1')
    return pnum

In [40]:
def get_rand_npod(n1 = 3, n2 = 3):

    num1 = randint(0, (10**n1-1))
    num2 = randint(0, (10**n2-1))
    pnum = str(num1).rjust(n1, '1') + '-' + str(num2).rjust(n2, '1')
    return pnum

In [41]:
def get_rand_date(start_date = '01.01.1940', stop_date = '01.01.2000'):
    d1, m1, y1 = map(int, start_date.split('.'))
    d2, m2, y2 = map(int, stop_date.split('.'))

    rand_y = str(randint(y1, y2))
    rand_m = str(randint(1, 12))
    rand_d = str(randint(1, 28))
    rand_date = rand_d.rjust(2, '0') + "." + rand_m.rjust(2, '0') + "." + rand_y

    return rand_date

# Функция генерации датфрейма с данными для N паспортов

In [42]:
def gen_pasp_data(n = 10):

  df_cities = get_cities()
  df_names = get_names()

  #keys = ['PSER', 'PNUM', 'VYD1', 'VYD2', 'VYD3', 'DVYD', 'NPOD', 'FAM1', 'FAM2', 'NAME', 'FNAM', 'BDAT','SX', 'BPL1', 'BPL2', 'BPL3']
  keys_descr = np.array([
      ['PSER', 'Серия паспорта'],
      ['PNUM', 'Номер паспорта'],
      ['VYD1', 'Паспорт выдан (1 строка)'],
      ['VYD2', 'Паспорт выдан (2 строка)'],
      ['VYD3', 'Паспорт выдан (3 строка)'],
      ['DVYD', 'Дата выдачи'],
      ['NPOD', 'Номер подразделения'],
      ['FAM1', 'Фамилия (1 строка)'],
      ['FAM2', 'Фамилия (2 строка)'],
      ['NAME', 'Имя'],
      ['FNAM', 'Отчество'],
      ['BDAT', 'Дата рождения'],
      ['SX', 'Пол'],
      ['BPL1', 'Место рождения (1 строка)'],
      ['BPL2', 'Место рождения (2 строка)'],
      ['BPL3', 'Место рождения (3 строка)']
  ])

  data = pd.DataFrame(columns = keys_descr[:,0])
  data.loc[0] = keys_descr[:,1]

  for i in range(n):

    pser = get_rand_pser()
    pnum = get_rand_pnum()
    npod = get_rand_npod()

    #vyd1, vyd2, vyd3 = get_rand_vyd()
    place = df_cities.sample(1)
    vyd1 = "Управление МВД России"
    vyd2 = "по г. " + place.city.iloc[0] + ", "
    vyd3 =  place.region_name.iloc[0]

    bdat = get_rand_date()
    bd, bm , by = bdat.split('.')

    dvyd = get_rand_date(start_date = "01.01." + str(int(by) + 14), stop_date = "01.01.2024")

    #fam1, fam2, name, fname, sx = get_rand_name()
    full_name = df_names.sample(1)
    fam1 = full_name.iloc[0,0]
    name = full_name.iloc[0,1]
    fnam = full_name.iloc[0,2]
    sx = ('муж.' if full_name.iloc[0,3] == "М" else 'жен.')

    #bpl1, bpl2, bpl3 = get_rand_bplace()
    place = df_cities.sample(1)
    bpl1 = "г. " + place.city.iloc[0] + ","
    bpl2 = place.region_name.iloc[0]

    data.loc[i+1] = [
         pser,
         pnum,
         vyd1,
         vyd2,
         vyd3, #'VYD3'
         dvyd,
         npod,
         fam1,
         "", #'FAM2'
         name,
         fnam,
         bdat,
         sx,
         bpl1,
         bpl2,
         "" #'BPL3'
    ]

  return data

# Функция сохранения docx в формате PNG

In [43]:
from spire.doc import *
from spire.doc.common import *

def print_png(path): #функция печати docx в PNG

  document = Document()

  # Load a Word file
  document.LoadFromFile(path + '.docx')

  # Loop through the pages in the document
  #for i in range(document.GetPageCount()): #перебор не будем делать, печатаем только первую страницу
    # Convert a specific page to bitmap image
  imageStream = document.SaveImageToStreams(
      #i,
      0,
      ImageType.Bitmap
      )
  # Save the bitmap to a PNG file
  with open(path + '.png','wb') as imageFile:
      imageFile.write(imageStream.ToArray())

  document.Close()

# Функции работы с PNG

In [44]:
def clean_passport_png(path, l = 135, u = 90, r = 660, d = 470): #функция обрезания самого паспорта
  with Image.open(path) as img:
    img.load()

  img_clean = img.crop((l, u, r, d))
  img_clean.save(path)

In [45]:
def clean_top_png(path, h = 60): #функция обрезания верха фотографии высотой h пикселей
  with Image.open(path) as img:
    img.load()

  img_clean = img.crop((0, h, img.width, img.height))
  img_clean.save(path)

In [46]:
def clean_bottom_png(path, h = 430): #функция обрезания низа фотографии (ниже h пикселей)
  with Image.open(path) as img:
    img.load()

  img_clean = img.crop((0, 0, img.width, h))
  img_clean.save(path)

580 - 465, 380 - 290 - координаты места под фотографию на шаблоне

In [47]:
#def put_photo(pasp_path, photo_path, l = 465, u = 290, r = 580, d = 380): #Функция добавления фотографии, точки left, up, right, down (lower)
def put_photo(pasp_path, photo_path, l = 465, u = 350, r = 580, d = 440): #Функция добавления фотографии, точки left, up, right, down (lower)
  with Image.open(pasp_path) as pasp:
    pasp.load()
  with Image.open(photo_path) as photo:
    photo.load()

  photo = photo.rotate(90)

  pasp.paste(
      photo.resize(( r - l, d - u)), # уменьшаем фото
       (l, u)
       )
  pasp.save(pasp_path)

In [48]:
def png_rand_rotate(pasp_path): #Функция поворота фотографии на случайный угол
  with Image.open(pasp_path) as pasp:
    pasp.load()

  #angle = randint(1, 3)*90 + randint(-3, 3) #случайный угол из (0, 90, 180, 270) +-3
  angle = 270
  pasp = pasp.rotate(angle, expand=True, fillcolor = 1 )

  pasp.save(pasp_path)

In [49]:
def png_rand_color(pasp_path): #Функция изменения цвета
  with Image.open(pasp_path) as pasp:
    pasp.load()

  if(randint(-1, 1) > 0):
    pasp = pasp.convert("L")

  pasp.save(pasp_path)

In [50]:
def png_blur(pasp_path): #Функция изменения цвета
  with Image.open(pasp_path) as pasp:
    pasp.load()

  pasp = pasp.filter(ImageFilter.GaussianBlur(1))

  pasp.save(pasp_path)

In [51]:
def png_grey(pasp_path): #Функция изменения цвета
  with Image.open(pasp_path) as pasp:
    pasp.load()

  pasp = pasp.convert("L")

  pasp.save(pasp_path)

In [52]:
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))



# Функция генерации датасета: PNG и соответствующего ему CSV

In [62]:
def make_dataset(datas):#функия генерации самого датасета (doc+json)
  n_samples = len(datas)
  PATH = os.getcwd()
  date = datetime.isoformat(datetime.now()).replace('-','_')[0:10]

  path_out = PATH + "/ds_pasports_" + date
  path_photo = PATH + "/photos"

  if os.path.exists(path_out) == False:
    os.mkdir(path_out)

  templates = ["/pasp_template8_1.docx", "/pasp_template8_2.docx","/pasp_template8_3.docx", "/pasp_template8_4.docx"]
  #templates = ["/pasp_template7_1.docx"]
  photos = os.listdir(path_photo)

  keys = datas.columns
  descriptions = datas.iloc[0]

  for sample_index in range(1, n_samples):

    templ_index = randint(0, len(templates)-1)
    #photo_index = randint(0, len(photos)-1)
    photo_index = sample_index

    doc = DocxTemplate(PATH + templates[templ_index])

    sample = datas.loc[sample_index]

    #context = {key : feature for key, feature in zip(keys, sample) }
    context = {key.lower() : feature for key, feature in zip(keys, sample) }  #внести в код
    context['pser'] = context['pser'][:2] + "  " + context['pser'][2:]        #внести в код
    doc.render(context)
    doc.save(f"{path_out}/{sample_index}.docx")

    print_png(f"{path_out}/{sample_index}")
    os.remove(f"{path_out}/{sample_index}.docx")

    put_photo(f"{path_out}/{sample_index}.png", path_photo + '/' + photos[photo_index])
    clean_passport_png(f"{path_out}/{sample_index}.png")
    #clean_top_png(f"{path_out}/{sample_index}.png")
    #clean_bottom_png(f"{path_out}/{sample_index}.png")
    #png_rand_color(f"{path_out}/{sample_index}.png")
    #png_grey(f"{path_out}/{sample_index}.png")
    #png_blur(f"{path_out}/{sample_index}.png")
    png_rand_rotate(f"{path_out}/{sample_index}.png")

    sample_json = {descr : feature for descr, feature in zip(descriptions, sample) }
    sample_json['Класс документа'] = "Паспорт"
    sample_json['Страна'] = "Российская Федерация"
    sample_json['Печать'] = "CV макет Test"
    json_to_file = json.dumps(sample_json, indent=4, ensure_ascii=False)

    # Writing to sample.json
    with open(f"{path_out}/{sample_index}.json", "w") as outfile:
        outfile.write(json_to_file)


  zip_filename = str(n_samples - 1) + 'sampl_dataset_pasports_' + date + ".zip"
  zipf = zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED)
  zipdir(path_out, zipf)
  zipf.close()

  files_to_del = os.listdir(path_out)
  for ftd in files_to_del:
    os.remove(path_out + "/" + ftd)

  os.rmdir(path_out)

  datas.to_csv("/content/data.csv")



# main()

In [ ]:
rand_pasp_data = gen_pasp_data(n = 10)

100%|██████████| 55.1M/55.1M [00:03<00:00, 14.9MB/s]

Extracting files...


In [ ]:
make_dataset(rand_pasp_data)

# Эта часть для генерации вопросника

In [64]:
def get_QA(datas: pd.DataFrame): #функция создания вопросника (дата в путь вписана вручную)
  datas.fillna("", inplace = True)
  datas_to_QA = datas.drop(columns = ['PSER','VYD2', 'VYD3', 'FAM2', 'BPL2', 'BPL3'])

  datas_to_QA['VYD1'] = datas['VYD1'] + " " + datas['VYD2'] + " " + datas['VYD3']
  datas_to_QA['BPL1'] = datas['BPL1'] + " " + datas['BPL2'] + " " + datas['BPL3']
  datas_to_QA['FAM1'] = datas['FAM1'] + " " + datas['FAM2']
  datas_to_QA['PNUM'] = datas['PSER'] + " " + datas['PNUM']

  datas_to_QA.iloc[0] = ['Серия и номер паспорта','Паспорт выдан','Дата выдачи','Номер подразделения','Фамилия','Имя','Отчество', 'Дата рождения','Пол','Место рождения']


  testQA_columns = ['data_index','question', 'answer', 'doc class', 'question type', 'answer bbox','split', 'image_path']
  testQA = []

  for sample_index in range(1, len(datas_to_QA)):
    for qw, anw in zip(datas_to_QA.iloc[0], datas_to_QA.iloc[sample_index]):
      testQA.append([sample_index, "Напиши " + qw, anw, 'Паспорт РФ', qw, "", 'test_clean', "\content\ds_pasports_2024_12_07\\" + str(sample_index) + '.png'])

  testQA = pd.DataFrame(testQA, columns = testQA_columns)
  testQA.to_csv("/content/10_samples_qwa.csv", sep=";", encoding="utf-8-sig", index=False)

  return testQA

In [2]:
%%script false
datas = pd.read_csv("/content/data (3).csv", index_col=0)
testQA = get_QA(datas)
make_dataset(datas)
datas.to_csv("/content/data.csv", sep=";", encoding="utf-8-sig", index=False)